In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [2]:
# Замените все, кроме букв и цифр, на пробелы
# salary_train.zip:
# https://d3c33hcgiwev3.cloudfront.net/_2a5a306cfe8235d3e65bddf5a5be6d0a_salary-train.zip?Expires=1628812800&Signature=WkxA4fTOy64jfadiljDngGAtdrF8I1cCF4u4FZZHXpAvfg~B~9sklzPYPSEIvcTCjBEzNgah7UGSoYRgAggwEZIPAP0S1NTKfbD5YfsOUoZ9QKWZkthNKA6pSy1u56wnbnvYDLf-amZW5LGL06ZUh7B00jl3VsK-w-Oh4fvYZM4_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A

salary_train_df = pd.read_csv('salary-train.csv').replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_train_df.shape

(60000, 4)

In [3]:
# Замените все, кроме букв и цифр, на пробелы 
salary_test_df = pd.read_csv('salary-test-mini.csv').replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_test_df.shape

(2, 4)

In [4]:
# Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'
salary_train_df['LocationNormalized'].fillna('nan', inplace=True)
salary_train_df['ContractTime'].fillna('nan', inplace=True)

salary_test_df['LocationNormalized'].fillna('nan', inplace=True)
salary_test_df['ContractTime'].fillna('nan', inplace=True)

In [5]:
# Приведите тексты к нижнему регистру (text.lower())
for col in salary_train_df.columns.drop(['SalaryNormalized']):
    salary_train_df[col] = salary_train_df[col].str.lower()
    salary_test_df[col] = salary_test_df[col].str.lower()

In [6]:
# Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
# Оставьте только те слова, которые встречаются хотя бы в 5 объектах 
# (параметр min_df у TfidfVectorizer).

vectorizer = TfidfVectorizer(min_df = 5)

X_train_vec = vectorizer.fit_transform(salary_train_df['FullDescription'])
X_test_vec = vectorizer.transform(salary_test_df['FullDescription']) 

In [7]:
# Примените DictVectorizer для получения 
# one-hot-кодирования признаков LocationNormalized и ContractTime

encoder = DictVectorizer(sparse = False)

X_train_categ = encoder.fit_transform(salary_train_df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = encoder.transform(salary_test_df[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [8]:
# Объедините все полученные признаки в одну матрицу "объекты-признаки". 
# Обратите внимание, что матрицы для текстов 
# и категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack

X_train = hstack([X_train_vec, X_train_categ])
X_test = hstack([X_test_vec, X_test_categ])
# целевая переменная
y_train = salary_train_df['SalaryNormalized']

In [9]:
# Обучите гребневую регрессию с параметрами alpha=1 и random_state=241

ridge = Ridge(alpha = 1, random_state = 241)
ridge.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

In [10]:
# прогнозы для двух примеров из файла salary-test-mini.csv

pred = ridge.predict(X_test)
for p in pred:
    print(round(p, 2))

56576.86
37136.3
